In [ ]:
from src.ga_api_package.ga_api_client import Repository, System, Tenant
import logging
import pandas as pd

# logging.basicConfig(level=logging.INFO)

In [ ]:
# create repository object
repo = Repository('https://rdlab-214.genie-analytics.com/api', 'api@default', 'api123!@#', ssl=False, burst=1, retry=0)
# create dataset
dset = await repo.set_dataset(13579, {
    'name':'example2',
    'series':'hour',
    'freq':300,
    'retainDepth':86400*7,
    'retainSize':10000000,
    'run':False
})
# delete all pipelines
await dset.del_all_pipelines()
# create pipeline
pipeline = {
    'scope': {
        'i-field': '@controller.home',
        'i-entry': [
            [True,'t:0:0:1',0]
        ],
        'mode': '%directional'
    },
    # 'bucket': [
    #     ['$distinctTuple', {
    #         'fields':['@flow.protocol'],
    #         'unwind':[False],
    #         'sizeLimit':256,
    #         'sortedBy':['$sum', {
    #             'field':'@flow.bytes'
    #         }],
    #         'sortedDir':1,
    #         'nullOpt':False
    #     }]
    # ],
    'metric': [
        ['$sum', {
            'field':'@flow.bytes'
        }]
    ]
}
pipe = await dset.set_pipeline(1001, {
    'name':'example',
    'pipeline': pipeline
})
# specify the time range
freq = '5T'
end = pd.Timestamp.now('+08:00').floor(freq) - pd.Timedelta(freq)
start = end - pd.Timedelta('48H')
dts = pd.date_range(start=start, end=end, freq=freq)

In [ ]:
# patch dataset data
df_patch = await dset.patch_data(dts)
df_patch

In [ ]:
df_patch[df_patch.isna().any(axis=1)]

In [ ]:
# poll dataset data
df_poll = await dset.poll_data(dts)
df_poll

In [ ]:
df_poll[df_poll.isna().any(axis=1)]

In [ ]:
# query pipeline data
df_query = await pipe.query_data(dts)
df_query

In [ ]:
df_query[df_query.isna().any(axis=1)]

In [ ]:
adhoc = await repo.set_adhoc(300, pipeline)

In [ ]:
df_adhoc = await adhoc.query_data(dts, 'hour')
df_adhoc

In [ ]:
df_adhoc[df_adhoc.isna().any(axis=1)]

In [ ]:
await repo.close()